In [1]:
from subgrounds import Subgrounds
import pandas as pd

sg = Subgrounds()
uniswap_v3 = sg.load_subgraph("https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3")

In [2]:
eth_usdc_005_pool = '0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640'
lp_fees = 0.05

In [3]:
pool_day_datas = uniswap_v3.Query.poolDayDatas(
    first = 1500,
    where={ 'pool': eth_usdc_005_pool},
    selection_set=[
        uniswap_v3.PoolDayData.date,
        uniswap_v3.PoolDayData.volumeUSD,
        uniswap_v3.PoolDayData.volumeToken0,
        uniswap_v3.PoolDayData.volumeToken1,
        uniswap_v3.PoolDayData.liquidity,
        uniswap_v3.PoolDayData.sqrtPrice,
        uniswap_v3.PoolDayData.tick,
    ]
)

In [4]:
pool_day_df = sg.query_df([
    pool_day_datas.date,
    pool_day_datas.volumeUSD,
    pool_day_datas.volumeToken0,
    pool_day_datas.volumeToken1,
    pool_day_datas.liquidity,
    pool_day_datas.sqrtPrice,
    pool_day_datas.tick,
])

In [5]:
pool_day_df['date'] = pd.to_datetime(pool_day_df['poolDayDatas_date'], unit='s')
print(pool_day_df.shape)
pool_day_df.tail()

(1124, 8)


,poolDayDatas_date,poolDayDatas_volumeUSD,poolDayDatas_volumeToken0,poolDayDatas_volumeToken1,poolDayDatas_liquidity,poolDayDatas_sqrtPrice,poolDayDatas_tick,date
1119,1716854400,3.124616e+08,3.124498e+08,81170.334739,15247759157542641034,1278678874938191088791505862304148,193789,2024-05-28
1120,1716940800,1.002871e+09,1.002387e+09,266365.328259,15601776730158281529,1291376875337734729360772501643465,193987,2024-05-29
1121,1717027200,4.082516e+08,4.083550e+08,108746.644083,1751185677919104221280,1294193769663909529613367257164644,194031,2024-05-30
1122,1717113600,5.233400e+08,5.234518e+08,139043.745647,17568531655938464393,1292001847940125115855171681495361,193997,2024-05-31
1123,1717200000,6.312590e+07,6.315155e+07,16706.535966,20654517611416424961,1285552206753172438155963628849727,193897,2024-06-01


In [24]:
swaps = uniswap_v3.Query.swaps(
    first = 300000,
    where={'pool': eth_usdc_005_pool},
    selection_set=[
        uniswap_v3.Swap.timestamp,
        uniswap_v3.Swap.transaction.id,
        uniswap_v3.Swap.transaction.gasUsed,
        uniswap_v3.Swap.transaction.gasPrice,
    ]
)

swaps_df = sg.query_df([
    swaps.timestamp,
    swaps.transaction.id,
    swaps.transaction.gasUsed,
    swaps.transaction.gasPrice,
])

In [25]:
swaps_df['timestamp'] = pd.to_datetime(swaps_df['swaps_timestamp'], unit='s')
swaps_df['date'] = swaps_df['timestamp'].dt.date
avg_gas_price_df = swaps_df.groupby('date').agg({'swaps_transaction_gasPrice': 'mean'}).reset_index()

In [26]:
swaps_df.tail()

,swaps_timestamp,swaps_transaction_id,swaps_transaction_gasUsed,swaps_transaction_gasPrice,timestamp,date
299900,1674153875,0x0b586d5c2d59f40c17785d3dc3f93c3942545e3be817...,310766,23703154311,2023-01-19 18:44:35,2023-01-19
299901,1626285930,0x0b586fcdb8103ffddf65d48405070d2278331555bb1f...,169774,39000000000,2021-07-14 18:05:30,2021-07-14
299902,1669100423,0x0b587161f7fd8e8d1052c6a08e40d6ea3aa0f224598a...,409228,11604753163,2022-11-22 07:00:23,2022-11-22
299903,1645285946,0x0b587207b05390466b373d7688ce113cbbde942453f6...,1200000,67958823992,2022-02-19 15:52:26,2022-02-19
299904,1699751927,0x0b5874502698f3783ebd534eea5aa5d66d66db8141ba...,251998,22311882261,2023-11-12 01:18:47,2023-11-12
...,...,...,...,...,...,...
299995,1647535142,0x0b59551ba3e988ea6b35e3fe0f821cc8ea652acf5cf1...,194385,136587625924,2022-03-17 16:39:02,2022-03-17
299996,1665073811,0x0b595747fd0e809a7b7b05893f8b61139a8cb50562e7...,263062,15272376380,2022-10-06 16:30:11,2022-10-06
299997,1680929399,0x0b595a3315bdc5dc5877f7123ad2640d84f29a310e58...,382629,18725526088,2023-04-08 04:49:59,2023-04-08
299998,1690228403,0x0b595af8aa834c6c1f9b7a06e56a74920c24ac0bdf3d...,357290,41069575942,2023-07-24 19:53:23,2023-07-24


In [35]:
merged_df = pd.merge(pool_day_df, avg_gas_price_df, left_on=pool_day_df['date'].dt.date, right_on='date', how='left')

# Drop the duplicate date column
merged_df.drop(columns=['date_y'], inplace=True)
merged_df.rename(columns={'date_x': 'date', 'swaps_transaction_gasPrice': 'avgGasPrice'}, inplace=True)

merged_df.tail()
merged_df.to_csv('./df.csv', index=False)

NameError: name 'pool_day_df' is not defined